In [1]:
import altair as alt 
alt.data_transformers.enable('json') # work-around to let Altair handle larger data sets

import pandas as pd
import numpy as np
import pycountry

# Preprocessing

In [2]:
winners = pd.read_csv('tdf_winners.csv')

In [3]:
# map country name to alpha3code 
winners['nationality'] = winners['nationality'].apply(lambda x : pycountry.countries.search_fuzzy(x)[0].alpha_3)
# grab only the year from date
winners['start_date'] = winners['start_date'].map(lambda x : int(x[:4]))
# add a column of ones, for sum aggregating in Altair
winners['Victories'] = np.ones(len(winners))

In [4]:
# remap columns names
winners = winners[['start_date', 'winner_name', 'stage_wins', 'stages_led', 'nationality', 'Victories']]
winners.columns=['Date', 'Name', 'Stage_wins', 'Stages_led', 'Nationality', 'Victories']
winners = winners.sort_values('Date', ascending=True)
winners

,Date,Name,Stage_wins,Stages_led,Nationality,Victories
0,1903,Maurice Garin,3,6,FRA,1.0
1,1904,Henri Cornet,1,3,FRA,1.0
2,1905,Louis Trousselier,5,10,FRA,1.0
3,1906,René Pottier,5,12,FRA,1.0
4,1907,Lucien Petit-Breton,2,5,FRA,1.0
...,...,...,...,...,...,...
101,2015,Chris Froome,1,16,GBR,1.0
102,2016,Chris Froome,2,14,GBR,1.0
103,2017,Chris Froome,0,15,GBR,1.0
104,2018,Geraint Thomas,2,11,GBR,1.0


In [5]:
### In the following cells we create a new aggregated dataframe per year and country
columns = [str(year) for year in range(1903, 2020)]
victory_by_year_country=pd.DataFrame(columns=columns)

for country in winners["Nationality"].unique():
    list_pays=[]
    for date in columns:
        list_pays.append(winners[(winners["Date"]<=int(date)) & (winners["Nationality"]==country)].count().values[0])
    
    tem_df=pd.DataFrame([list_pays],columns=columns)
    tem_df.index=[country]
    victory_by_year_country=victory_by_year_country.append(tem_df)

In [6]:
list_winners_by_country = []
for country in winners["Nationality"].unique():
    list_winners_by_country.append(winners[(winners["Date"]<=2019) & (winners["Nationality"]==country)][["Name","Date"]].values)

In [7]:
winners_by_country=pd.DataFrame(list_winners_by_country, columns=["victories"],index=winners["Nationality"].unique())

In [8]:
victory_by_year_country["victories"]=winners_by_country

# we need this file to map each country with their corresponding id for the choropleth map:
country_info = pd.read_json('https://raw.githubusercontent.com/MariaKokshaikina/any-data/main/all_countries.json')
victory_by_year_country['id'] = victory_by_year_country.index.map(lambda x: country_info[country_info["alpha3Code"]==x]['numericCode'].values[0])

victory_by_year_country.reset_index(inplace=True)
victory_by_year_country

,index,1903,1904,1905,1906,1907,1908,1909,1910,1911,...,2012,2013,2014,2015,2016,2017,2018,2019,victories,id
0,FRA,1,2,3,4,5,6,6,7,8,...,36,36,36,36,36,36,36,36,"[[Maurice Garin, 1903], [Henri Cornet, 1904], ...",250.0
1,LUX,0,0,0,0,0,0,1,1,1,...,5,5,5,5,5,5,5,5,"[[François Faber, 1909], [Nicolas Frantz, 1927...",442.0
2,BEL,0,0,0,0,0,0,0,0,0,...,18,18,18,18,18,18,18,18,"[[Odile Defraye, 1912], [Philippe Thys, 1913],...",56.0
3,ITA,0,0,0,0,0,0,0,0,0,...,9,9,10,10,10,10,10,10,"[[Ottavio Bottecchia, 1924], [Ottavio Bottecch...",380.0
4,CHE,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,"[[Ferdinand Kübler, 1950], [Hugo Koblet, 1951]]",756.0
5,ESP,0,0,0,0,0,0,0,0,0,...,12,12,12,12,12,12,12,12,"[[Federico Bahamontes, 1959], [Luis Ocaña, 197...",724.0
6,NLD,0,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,"[[Jan Janssen, 1968], [Joop Zoetemelk, 1980]]",528.0
7,USA,0,0,0,0,0,0,0,0,0,...,10,10,10,10,10,10,10,10,"[[Greg LeMond, 1986], [Greg LeMond, 1989], [Gr...",840.0
8,IRL,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,"[[Stephen Roche, 1987]]",372.0
9,DNK,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,"[[Bjarne Riis, 1996]]",208.0


# Visualisation

In [9]:
from altair.expr import datum
from vega_datasets import data


### Parameters; filter,slider, country selection
slider = alt.binding_range(min=1903, max=2019, step=1, name='year')
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider, init={'year': 2006})

click_selection = alt.selection_single(
    empty='none',
    fields=['index'],
    name='click_selection',init={"index":"FRA"}

)

selection = alt.selection_multi(fields=['index'])

filter_dropdown = alt.binding_select(options=['Stage_wins', 'Stages_led', 'Victories'], name='filter')
filter_select = alt.selection_single(fields=['filter'], bind=filter_dropdown, name="filter", init={'filter': 'Stage_wins'})


### Choropleth map
countries = alt.topo_feature(data.world_110m.url, 'countries')

country_map=alt.Chart(countries).mark_geoshape()\
    .project('equirectangular')\
    .transform_lookup(
        lookup='id',
        from_=alt.LookupData(victory_by_year_country, 'id', columns+["index"]+["victories"])
    ).transform_fold(
        columns, as_=['year', 'tdf_victories']
    ).transform_calculate(
        year='parseInt(datum.year)',
        tdf_victories='isValid(datum.tdf_victories) ? datum.tdf_victories : 0'

    ).encode(
        color = alt.condition(
            'datum.tdf_victories > 0',
            alt.Color('tdf_victories:Q', scale=alt.Scale(scheme='blues')),
            alt.value('#dbe9f6'),        
        ),    
        tooltip=["year:Q",'tdf_victories:Q',"index:O","victories:N"],

    ).add_selection(
        select_year
    ).transform_filter(
        select_year
    ).add_selection(
        click_selection
    ).properties(title='Tour de France winners per country',
                width=700,
                height=400)


### Race chart
source = winners    
source.rename(columns = {'Date':'year','Nationality':"index"}, inplace = True)


horizontal_bar_chart = alt.Chart(source
    ).transform_fold(
        ['Stage_wins', 'Stages_led', 'Victories'], as_=['filter', 'value']
    ).transform_filter(
        filter_select
    ).transform_aggregate(
        nb_wins = 'sum(value)',
        groupby=['Name']
    ).mark_bar().encode(
        x=alt.X('nb_wins:Q'),
        y=alt.Y("Name:O", sort='-x'),
        color=alt.Color('nb_wins:Q', legend=None),
        facet=alt.Facet('index:N', title="Country:"),
        tooltip=[alt.Tooltip('Name:O', title="Name"),
                 alt.Tooltip('nb_wins:Q', title="Wins")],
    ).transform_lookup(
        lookup='Name',
        from_=alt.LookupData(source, 'Name', ["index","year"])
    ).transform_filter(
         "datum.index == click_selection.index"
    ).transform_filter(
         "datum.year <= year.year"
    ### TOP K ITEM ###
    #).transform_window(
    #    rank='rank(sum_values)',
    #    sort=[alt.SortField('nb_wins', order='descending')]
    #).transform_filter(
    #    (alt.datum.rank < 10)
    ### TOP K ITEM END ###
    ).properties(width=700,
                height=300
    ).add_selection(
        filter_select
)


### Final design
alt.vconcat(country_map,horizontal_bar_chart.interactive()).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.VConcatChart(...)